In [ ]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# Function to get latitude and longitude of a location
def get_lat_lon(api_key, location):
    geo_url = f"https://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={api_key}"
    try:
        response = requests.get(geo_url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data:
            return data[0]["lat"], data[0]["lon"]
        else:
            print("Location not found.")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching location data: {e}")
        return None, None

# Function to get current weather data using OpenWeatherMap API
def get_weather_data(api_key, lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return {
            "Max_Temp": data["main"]["temp"],
            "Rainfall": data.get("rain", {}).get("1h", 0),
            "Relative_Humidity": data["main"]["humidity"],
            "Wind_Speed": data["wind"]["speed"],
            "Cloud_Coverage": data["clouds"]["all"],
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

# Load dataset
df = pd.read_csv("C:/Users/incha/Downloads/FloodPrediction.csv")

# Fill missing values in 'Flood?' column with 0 (No Flood)
df['Flood?'] = df['Flood?'].fillna(0)

# Selecting relevant features
features = ["Max_Temp", "Rainfall", "Relative_Humidity", "Wind_Speed", "Cloud_Coverage"]
X = df[features]

# Target variables
y_flood = df["Flood?"]

# Encoding categorical target variable
label_encoder = LabelEncoder()
y_flood = label_encoder.fit_transform(y_flood)

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y_flood, test_size=0.2, random_state=42)

# Normalizing data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/harsh/Downloads/FloodPrediction.csv'

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)


Epoch 1/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9080 - loss: 0.2603 - val_accuracy: 0.9465 - val_loss: 0.1430
Epoch 2/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9367 - loss: 0.1642 - val_accuracy: 0.9498 - val_loss: 0.1380
Epoch 3/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9377 - loss: 0.1635 - val_accuracy: 0.9492 - val_loss: 0.1401
Epoch 4/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9403 - loss: 0.1582 - val_accuracy: 0.9480 - val_loss: 0.1400
Epoch 5/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9433 - loss: 0.1481 - val_accuracy: 0.9483 - val_loss: 0.1389
Epoch 6/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9416 - loss: 0.1509 - val_accuracy: 0.9462 - val_loss: 0.1408
Epoch 7/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9386 - loss: 0.1547 - val_accuracy: 0.9465 - val_loss: 0.1412
Epoch 8/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9443 - loss: 0.1502 - val_accuracy: 0.

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step
Model Accuracy: 94.35%


In [ ]:
# Save the trained model
model.save(r"C:\Users\harsh\OneDrive\Desktop\flood_prediction\flood_prediction_model.h5")

In [ ]:
def predict_flood(api_key):
    location = input("Enter city or state name: ")
    lat, lon = get_lat_lon(api_key, location)
    if lat is not None and lon is not None:
        print(f"Location: {location}, Latitude: {lat}, Longitude: {lon}")
        weather_data = get_weather_data(api_key, lat, lon)
        if weather_data:
            weather_df = pd.DataFrame([weather_data])
            weather_scaled = scaler.transform(weather_df)
            prediction = model.predict(weather_scaled)[0][0]
            if prediction > 0.7:
                flood_risk = "High Risk Flood Area"
            elif prediction > 0.4:
                flood_risk = "Moderate Risk Area"
            else:
                flood_risk = "Flood Free Area"
            print(f"Flood Prediction for {location}: {flood_risk}")
        else:
            print("Failed to retrieve weather data.")
    else:
        print("Failed to fetch latitude and longitude.")

In [ ]:
api_key = "e3a90990b56034cdb755847b6439abae"
predict_flood(api_key)


Location: mysuru, Latitude: 12.3051828, Longitude: 76.6553609


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Cloud_Coverage
- Max_Temp
- Rainfall
- Relative_Humidity
- Wind_Speed
Feature names seen at fit time, yet now missing:
- Depth
- Latitude
- Location
- Longitude


In [ ]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import joblib

# Load the trained flood prediction model (.h5 file)
model = load_model("C:/Users/harsh/OneDrive/Desktop/flood_prediction/flood_prediction_model.h5")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_predictionscaler.pkl")

def get_lat_lon(api_key, location):
    """Fetch latitude and longitude for a given location."""
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={api_key}"
    response = requests.get(url).json()

    if response and isinstance(response, list) and len(response) > 0:
        lat = response[0]['lat']
        lon = response[0]['lon']
        return lat, lon
    return None, None

def get_weather_forecast(api_key, lat, lon):
    """Fetch 10-day weather forecast data from OpenWeather API."""
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&cnt=10&appid={api_key}"
    response = requests.get(url).json()

    if "list" in response:
        forecast_data = []
        for day in response["list"]:
            forecast_data.append({
                "Max_Temp": day["main"]["temp_max"],  # Max temperature
                "Min_Temp": day["main"]["temp_min"],  # Min temperature
                "Rainfall": day.get("rain", {}).get("3h", 0),  # Rainfall (default 0 if missing)
                "Relative_Humidity": day["main"]["humidity"],  # Humidity
                "Wind_Speed": day["wind"]["speed"],  # Wind speed
                "Cloud_Coverage": day["clouds"]["all"],  # Cloud coverage
                "Latitude": lat,  # Add Latitude
                "Longitude": lon,  # Add Longitude
                "Depth": np.random.uniform(1, 10),  # Simulated depth (If your model needs it)
                "Location": hash(location) % 1000  # Encode location as a number (If necessary)
            })
        return forecast_data
    return None

def predict_flood(api_key):
    """Predict flood risk for the next 10 days using the trained model."""
    location = input("Enter city or state name: ")
    lat, lon = get_lat_lon(api_key, location)

    if lat is not None and lon is not None:
        print(f"\n📍 Location: {location}, Latitude: {lat}, Longitude: {lon}")

        weather_forecast = get_weather_forecast(api_key, lat, lon)
        if weather_forecast:
            # Convert forecast data to DataFrame
            weather_df = pd.DataFrame(weather_forecast)

            print("📊 Model Input Data Before Scaling:")
            print(weather_df.head())  # Debugging print

            # Scale the weather data
            weather_scaled = scaler.transform(weather_df)

            print("📊 Scaled Data Before Prediction:")
            print(weather_scaled[:5])  # Debugging print

            # Predict flood risk for the next 10 days
            predictions = model.predict(weather_scaled).flatten()

            # Interpret predictions
            flood_predictions = []
            for i, prediction in enumerate(predictions):
                if prediction > 0.7:
                    flood_risk = "High Risk Flood Area"
                elif prediction > 0.4:
                    flood_risk = "Moderate Risk Area"
                else:
                    flood_risk = "Flood Free Area"

                flood_predictions.append(f"Day {i+1}: {flood_risk}")

            # Print predictions
            print("\n🌊 Flood Prediction for the Next 10 Days:")
            for pred in flood_predictions:
                print(pred)
        else:
            print("❌ Failed to retrieve 10-day weather forecast.")
    else:
        print("❌ Failed to fetch latitude and longitude.")

# Run the flood prediction function
API_KEY = "e3a90990b56034cdb755847b6439abae"
predict_flood(API_KEY)



📍 Location: Mumbai, Latitude: 19.0785451, Longitude: 72.878176
📊 Model Input Data Before Scaling:
   Max_Temp  Min_Temp  Rainfall  Relative_Humidity  Wind_Speed  \
0     31.28     27.78         0                 32        6.48   
1     29.20     27.28         0                 37        2.90   
2     26.04     26.04         0                 48        3.60   
3     25.24     25.24         0                 49        3.76   
4     25.69     25.69         0                 42        2.79   

   Cloud_Coverage   Latitude  Longitude     Depth  Location  
0               3  19.078545  72.878176  5.638142       119  
1               2  19.078545  72.878176  8.988548       119  
2               0  19.078545  72.878176  7.291736       119  
3               0  19.078545  72.878176  7.697227       119  
4               0  19.078545  72.878176  9.211237       119  


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Cloud_Coverage
- Max_Temp
- Min_Temp
- Rainfall
- Relative_Humidity
- ...


In [ ]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import joblib
from datetime import datetime, timezone, timedelta
# Load the trained flood prediction model (.h5 file)
model = load_model("C:/Users/harsh/OneDrive/Desktop/flood_prediction/flood_prediction_model.h5")

# Load the pre-trained scaler
scaler = joblib.load("C:/Users/harsh/OneDrive/Desktop/flood_prediction/scaler.pkl")

def get_lat_lon(api_key, location):
    """Fetch latitude and longitude for a given location."""
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={api_key}"
    response = requests.get(url).json()

    if response and isinstance(response, list) and len(response) > 0:
        lat = response[0]['lat']
        lon = response[0]['lon']
        return lat, lon
    return None, None

def get_weather_forecast(api_key, lat, lon):
    """Fetch 10-day weather forecast data from OpenWeather API."""
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&units=metric&cnt=10&appid={api_key}"
    response = requests.get(url).json()

    if "list" in response:
        forecast_data = []
        for day in response["list"]:
            timestamp = day["dt"]
            date = datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d')  # Fix DeprecationWarning

            forecast_data.append({
                "Date": date,
                "Max_Temp": day["main"]["temp_max"],  
                "Min_Temp": day["main"]["temp_min"],  
                "Rainfall": day.get("rain", {}).get("3h", 0),  
                "Relative_Humidity": day["main"]["humidity"],  
                "Wind_Speed": day["wind"]["speed"],  
                "Cloud_Coverage": day["clouds"]["all"],  
                "Latitude": lat,  # Add Latitude
                "Longitude": lon,  # Add Longitude
                "Depth": np.random.uniform(1, 10),  # Simulated depth (If your model needs it)
                "Location": hash(location) % 1000  # Encode location as a number (If necessary)
            })
        return forecast_data
    return None

def predict_flood(api_key):
    """Predict flood risk for the next 10 days using the trained model."""
    location = input("Enter city or state name: ")
    lat, lon = get_lat_lon(api_key, location)

    if lat is not None and lon is not None:
        print(f"\n📍 Location: {location}, Latitude: {lat}, Longitude: {lon}")

        weather_forecast = get_weather_forecast(api_key, lat, lon)
        if weather_forecast:
            # Convert forecast data to DataFrame
            weather_df = pd.DataFrame(weather_forecast)

            # Print expected vs actual feature names
            expected_columns = scaler.feature_names_in_
            print("\n✅ Expected Features:", expected_columns)
            print("\n✅ Actual Features in Weather Data:", weather_df.columns.tolist())

            # Ensure feature order matches the trained model
            weather_df = weather_df[expected_columns]

            print("\n📊 Model Input Data Before Scaling:")
            print(weather_df.head())  

            # Scale the weather data
            weather_scaled = scaler.transform(weather_df)

            print("\n📊 Scaled Data Before Prediction:")
            print(weather_scaled[:5])  

            # Predict flood risk for the next 10 days
            predictions = model.predict(weather_scaled).flatten()

            # Interpret predictions
            flood_predictions = []
            for i, prediction in enumerate(predictions):
                if prediction > 0.7:
                    flood_risk = "High Risk Flood Area"
                elif prediction > 0.4:
                    flood_risk = "Moderate Risk Area"
                else:
                    flood_risk = "Flood Free Area"

                flood_predictions.append(f"Day {i+1}: {flood_risk}")

            # Print predictions
            print("\n🌊 Flood Prediction for the Next 10 Days:")
            for pred in flood_predictions:
                print(pred)
        else:
            print("❌ Failed to retrieve 10-day weather forecast.")
    else:
        print("❌ Failed to fetch latitude and longitude.")

# Run the flood prediction function
API_KEY = "c5615a8df07bf39f21e624ffe731f157"
predict_flood(API_KEY)



📍 Location: Mumbai, Latitude: 19.0785451, Longitude: 72.878176

✅ Expected Features: ['Latitude' 'Longitude' 'Depth' 'Location']

✅ Actual Features in Weather Data: ['Date', 'Max_Temp', 'Min_Temp', 'Rainfall', 'Relative_Humidity', 'Wind_Speed', 'Cloud_Coverage', 'Latitude', 'Longitude', 'Depth', 'Location']

📊 Model Input Data Before Scaling:
    Latitude  Longitude     Depth  Location
0  19.078545  72.878176  2.524066       119
1  19.078545  72.878176  2.607219       119
2  19.078545  72.878176  4.251756       119
3  19.078545  72.878176  2.359132       119
4  19.078545  72.878176  1.689561       119

📊 Scaled Data Before Prediction:
[[-1.48015717 -0.79502645 -0.73590223 -1.5190145 ]
 [-1.48015717 -0.79502645 -0.73469005 -1.5190145 ]
 [-1.48015717 -0.79502645 -0.71071646 -1.5190145 ]
 [-1.48015717 -0.79502645 -0.7383066  -1.5190145 ]
 [-1.48015717 -0.79502645 -0.74806741 -1.5190145 ]]


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_9" is incompatible with the layer: expected axis -1 of input shape to have value 5, but received input with shape (10, 4)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(10, 4), dtype=float32)
  • training=False
  • mask=None